# 1. Exercícios

## 1.1 Organize o código do módulo 4 em funções
##### A estratégia para esse exercício é sequenciar o código em forma de ETL

In [ ]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import plotly.express as px
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from matplotlib import pyplot as plt
from matplotlib import gridspec #Para fazer os dashs
#Foram adicionados por conta de erro de renderização no mapa (ex1.10), utilizando o VS Code. Caso sua IDE não tenha o erro, não é necessário
#Erro: No renderer could be found for mimetype "application/vnd.plotly.v1+json"
import plotly.io as pio
pio.renderers.default = "notebook_connected"
from geopy.geocoders import Nominatim
import ipywidgets as widgets
from ipywidgets import fixed
import time
from multiprocessing import Pool


def coleta_dados(path):
    data = pd.read_csv(path)
    return data

#Definir a função que cria o mapa com o botão
def update_map(df, disponivel, ano_renovacao, waterfront):

    df = data[(data['date'] <= disponivel) & (data['yr_renovated'] >= ano_renovacao) &  (data['waterfront'] == waterfront).copy()]

    fig = plt.figure(figsize = (12,12))
    specs = gridspec.GridSpec(ncols = 2, nrows = 2, figure = fig)

    ax1 = fig.add_subplot(specs[0,:])
    ax2 = fig.add_subplot(specs[1,0])
    ax3 = fig.add_subplot(specs[1,1])
    by_year = data[['id', 'year']].groupby( 'year').sum().reset_index()
    ax1.bar( by_year['year'], by_year['id'] )

    by_day = data[['id', 'date']].groupby( 'date').mean().reset_index()
    ax2.plot( by_day['date'], by_day['id'] )
    ax2.set_title( 'title: Média de preço por dia' )

    by_week_of_year = data[['id', 'year_week']].groupby( 'year_week').mean().reset_index()
    ax3.bar( by_week_of_year['year_week'], by_week_of_year['id'] )
    ax3.set_title( 'title: Média de preço por semana do ano' )
            
    plt.xticks( rotation=60 )

    return widgets.interactive( update_map, df=fixed( data ), disponivel = bt_data_disponivel, ano_renovacao = bt_ano_renovacao,  waterfront = bt_waterfront)

def processa_dados(data):
    data['level'] = data['price'].apply(lambda x: 0 if (x >=0) & (x <=321950)  else
                                              1 if (x > 321950) & (x <=450000) else
                                              2 if (x > 450000) & (x <=645000) else
                                              3)

    data['size'] = data['sqft_living'].apply(lambda x: 0 if (x >= 0) & (x <= 1427) else
                                                    1 if (x > 1427) & (x <= 1910) else
                                                    2 if (x > 1910) & (x <= 2550) else
                                                    3)
    data['year'] = pd.to_datetime( data['date'] ).dt.strftime( '%Y' )
    data['date'] = pd.to_datetime( data['date'] ).dt.strftime( '%Y-%m-%d' )
    data['year_week'] = pd.to_datetime( data['date'] ).dt.strftime( '%Y-%U' )
    
    return data

def carrega_dados(data):
    
    ####################################
    # Criar botões e o mapa interativo #
    ####################################

    #Botão level
    data['level'] = data['price'].apply(lambda x:   0 if x <= 312950 else
                                                    1 if (x > 321950) & (x <= 450000) else
                                                    2 if (x > 450000) & (x <= 645000) else
                                                    3 )  

    #Botão vista pra água
    data['is_waterfront'] = data['waterfront'].apply(lambda x: 'yes' if x == 1 else 'no')
    bt_waterfront = widgets.Dropdown(  options     = data['is_waterfront'].unique().tolist(),
                                        value       = 'no',
                                        disable     = False,
                                        description = 'Vista para água?',
                                        style={'description_width': 'initial'}
                                    )


    #Botão tamanho mínimo da sala de estar
    bt_lv_room = widgets.FloatSlider(
                    value = 2000.0,
                    min = data['sqft_living'].min(),
                    max = data['sqft_living'].max(),
                    step = 0.1,
                    description = 'Tamanho mínimo da sala:',
                    disabled = False,
                    orientation = 'horizontal',
                    style={'description_width': 'initial'}
    )

    #Botão número mínimo de banheiros
    bt_bathrooms = widgets.IntSlider(
                    value = data['bathrooms'].min(),
                    min = data['bathrooms'].min(),
                    max = data['bathrooms'].max(),
                    step = 1,
                    description = 'Número mínimo de banheiros:',
                    disabled = False,
                    style={'description_width': 'initial'}
    )

    #Botão valor máximo do preço
    bt_price = widgets.FloatSlider(
                    value = 3300000,
                    min = data['price'].min(),
                    max = data['price'].max(),
                    step = 0.1,
                    description = 'Preço máximo do imóvel:',
                    disabled = False,
                    style={'description_width': 'initial'}
    )

    #Botão tamanho máximo área do porão
    bt_basement = widgets.IntSlider(
                    value = 1000,
                    min = data['sqft_basement'].min(),
                    max = data['sqft_basement'].max(),
                    step = 1,
                    description = 'Tamanho máximo do porão:',
                    disabled = False,
                    style={'description_width': 'initial'}
    )

    #Botão ano de condição
    bt_condition = widgets.IntSlider(
                    value = data['condition'].min(),
                    min = data['condition'].min() ,
                    max = data['condition'].max(),
                    step = 1,
                    description = 'Condição do imóvel:',
                    disabled = False,
                    style={'description_width': 'initial'}
    )

    #Botão ano de construção
    bt_yr_built = widgets.IntSlider(
                    value = data['yr_built'].min(),
                    min = data['yr_built'].min(),
                    max = data['yr_built'].max(),
                    step = 1,
                    description = 'Ano de construção:',
                    disabled = False,
                    style={'description_width': 'initial'}
    )

    #Criar o mapa
    def func_housesmap(df, waterfront, price, lv_room, bathrooms, basement, condition, yr_built):

        houses = df[(df['is_waterfront'] == waterfront) & 
                    (df['price'] <= price) & 
                    (df['sqft_living'] >= lv_room) & 
                    (df['bathrooms'] >= bathrooms) & 
                    (df['price'] <= price) & 
                    (df['sqft_basement'] <= basement) & 
                    (df['condition'] >= condition) & 
                    (df['yr_built'] >= yr_built)][['id', 'lat','long', 'price', 'level']]          
        
        fig = px.scatter_mapbox( houses, 
                                lat = 'lat',
                                lon = 'long',
                                color = 'level',
                                size = 'price',
                                color_continuous_scale = px.colors.cyclical.IceFire,
                                size_max = 15,
                                zoom = 8
        )
        fig.update_layout(mapbox_style = 'open-street-map')
        fig.update_layout(height = 600, margin = {'r':0, 't':0, 'l':0, 'b':0})
        fig.show()
            
    #Plotar o mapa com os filtros
    widgets.interact(func_housesmap, df = fixed(data), waterfront = bt_waterfront,
                                                    price = bt_price,
                                                    lv_room = bt_lv_room,
                                                    bathrooms = bt_bathrooms,
                                                    basement = bt_basement,
                                                    condition = bt_condition,
                                                    yr_built = bt_yr_built
    )


    ####################################
    # Criar botões e o dash interativo #
    ####################################

    #Criar o botão de disponibilidade a partir da data selecionada para o mapa
    bt_data_disponivel = widgets.SelectionSlider( options=data['date'].sort_values().unique().tolist(),
                                                value = '2014-12-01',
                                                description = 'Disponivel',
                                                continuous_update=False,
                                                orientation='horizontal',
                                                readout=True,
                                                style={'description_width': 'initial'}
    )

    #Botão para ano de renovação
    bt_ano_renovacao = widgets.IntSlider( value = data['yr_renovated'].min(),
                                        min = data['yr_renovated'].min(),
                                        max = data['yr_renovated'].max(),
                                        step = 1,
                                        description = 'Ano de renovação:',
                                        disabled = False,
                                        style={'description_width': 'initial'}
    )

    #Botão vista pra água
    data['is_waterfront'] = data['waterfront'].apply(lambda x: 'yes' if x == 1 else 'no')
    bt_waterfront = widgets.Dropdown(  options     = data['is_waterfront'].unique().tolist(),
                                        value       = 'no',
                                        disable     = False,
                                        description = 'Vista para água?',
                                        style={'description_width': 'initial'}
                                    )

    return None

if __name__ == '__main__':

    data_raw = coleta_dados('datasets/kc_house_data.csv')

    data_processing = processa_dados(data_raw)

    data_load = carrega_dados (data_processing)

    mostra_dash = update_map




In [ ]:
cols = ['placeID', 'osm_type','country', 'country_code']
df = data.head()
df1 = coleta_geodados(df, cols)
df1

## 1.2 Refaça o gráfico e o dash com o Streamlit